## Experiment 2

Simulation, visulization, and shifting of superposition of a single qubit as noise patterns on a square canvas.
This notebook is for further experimentation and adjustments within the scripts.

### Including all modules

In [1]:
# Qiskit for Quantum computation
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.providers.aer.noise import NoiseModel, QuantumError, ReadoutError, pauli_error

# PILlow for image generation
from PIL import Image, ImageColor

# Handy math libraries
import numpy as np
import math

# For saving files in current directory
import os

In [2]:
# Simulate Quantum computation

def run_qc(circuit, backend, output, n_shots):

    # -------------------------------------------------------------------------------
    
    # Build basic bit-flip error noise model

    # Example error probabilities
    p_reset = 0.03
    p_meas = 0.1
    p_gate1 = 0.05

    # QuantumError objects
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    # Add errors to noise model
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
    noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
    noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
    noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])
    
    # -------------------------------------------------------------------------------
    
    # Execution and options
    
    # Load simulator (Aer)
    backend_simulate = Aer.get_backend('aer_simulator')
    
    # Execute on Aer
    if (backend == 'sim'):
        run = execute(circuit,
                      backend_simulate,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()
    
    # Execute on Aer + noise model
    if (backend == 'sim_noise'):
        run = execute(circuit,
                      backend_simulate,
                      noise_model=noise_bit_flip,
                      shots = n_shots,
                      memory = True).result()
        if (output == 'count'):
            out = run.get_counts()
        if (output == 'memory'):
            out = run.get_memory()

    return out

In [3]:
# Map output values linear accoring to input values of x

def lin_map(x, in_min, in_max, out_min, out_max):
    if in_min == in_max:
        return out_max / 2
    else:
        return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

In [4]:
# Generate output image

def img_generating(data_vals, path):
    a = int(len(data_vals) ** (1/2))
    b = int(len(data_vals) ** (1/2))
    c = int(len(data_vals[0]))
    d = int(len(data_vals[0][0]))
    img = Image.new('RGB', (a * c, b * d))

    for i in range(a):
        for j in range(b):
            for k in range(c):
                for l in range(d):
                    color = int(data_vals[j + (a * i)][k][l] * 255) # [npatch index][npatch col][npatch row]
                    img.putpixel(((k + (j * c)), l + (i * d)), (color, color, color)) # (x, y, rgb color)

    #img.show()
    img.save(path)

In [5]:
# Generate episodic image of radiant-regulated noise pattern

def sample_noise(phase, size, n_frames, path):
    for m in range(n_frames):
        data_out = []
        
        # Quantum circuit
        qc = QuantumCircuit(1)
        qc.h(0)
        qc.ry(math.pi * phase, 0)
        qc.measure_all()

        # Measurements
        result = run_qc(qc, 'sim', 'memory', size ** 2)
        result = np.reshape(result, (size, size)).astype(float)
        data_out.append(result)
        
        # Image assembling
        img_generating(data_out, path)

### Execution of functions with different iterative variations

In [9]:
# Single exection of sample_noise with certain phase

phase = 1.0
sidelength = 20

phase_as_str = str(round(phase, 2)).replace('.', 'pt')
sample_noise(phase, sidelength, n_frames, f'range_{phase_as_str}_{sidelength}x{sidelength}px.png')

In [7]:
# Iterative exection of sample_noise with a linear range 

p_min = 0.0
p_max = 2.0
samples = 40
sidelength = 20

# multiple frames per execution for potential animations
n_frames = 1

# Sampling from linear space
phases = np.linspace(p_min, p_max, num=samples, endpoint=True)

for i in range(len(phases)):
    phase_as_str = str('{:.2f}'.format(round(phases[i], 2))).replace('.', 'pt')
    sample_noise(phases[i], sidelength, n_frames, f'range_{phase_as_str}_{sidelength}x{sidelength}px.png')